# PMR3401 - Mecânica Computacional para Mecatrônica
## Exercício-Programa 1
João Rodrigo Windisch Olenscki ($\texttt{10773224}$) </br>
Lui Damianci Ferreira ($\texttt{10770579}$)

### Equações, constantes e condições iniciais
 $$ m_{total}\ddot{x} + m_1 L (\theta^2 \cos{\theta} + \ddot{\theta}\sin{\theta}) = -F + F_1 \cos{\theta}$$
 
 $$ m_1 \ddot{x} L \sin{\theta} + m_1 \ddot{\theta} L^2 + 2 I \omega \dot{\theta} = 0 $$
 
 Constantes:
 
 $m_{total}=1939$ kg </br>
 $L=2.95$ m </br>
 $I = 1$ kg m² </br>
 $\omega=10$ rad/s </br>
 
 Forças no eixo: </br>
 1. Se tem tração no eixo: $F = \mu m g$ </br>
 2. Se não tem tração no eixo: $F=\beta m g$ </br>
 
 * $\mu=0.42$: coeficiente de atrito na roda quando há **rolamento** </br>
 * $\beta=0.02$: coeficiente de atrito da roda quando há **deslizamento**
 
 Obs.: nestas forças o termo de massa $m$ corresponde a **massa do eixo**, sendo uma fração da massa total (dependendo da distribuição inputada) </br>
 Obs².: eixos tracionados rolam e eixos não-tracionados deslizam

### Equações desacopladas

$$\ddot{\theta} = \left(\frac{\sin{\theta}}{m_1 L \sin^{2}{\theta} - m_{total} L} \right) \cdot \left[\frac{2 m_{total} I \omega \dot{\theta}}{m_1 L \sin{\theta}} - m_1 L \theta^2  \cos{\theta} - F + F_1 \cos{\theta} \right] $$

$$\ddot{x} = -\frac{m_1 L^2 \ddot{\theta} + 2 I \omega \dot{\theta}}{m_1 L \sin{\theta}} $$

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
# Parametros estéticos do matplotlib
plt.rcParams['mathtext.fontset'] = 'cm'
plt.rcParams['font.family'] = 'STIXGeneral'
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (10, 5),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
plt.rcParams.update(params)
# usar latex no matplotlib: https://stackoverflow.com/questions/41453109/how-to-write-your-own-latex-preamble-in-matplotlib
import os

# cores para cada variável
colors = plt.get_cmap('tab20b').colors
cores = {'theta' : colors[0],
         'thetap': colors[10],
         'xp'    : colors[4],
         'xpp'   : colors[13]
}


def create_folder(folder_name, path = os.getcwd()):
    '''
    Função que cria uma pasta em um determinado diretório. Criada com o objetivo de
    ser usada junto com as funções de plotagem para salvar as figuras
    
    Parameters
    ----------
    folder_name: str
        nome da pasta a ser criada
    path: str (default=os.getcwd(), diretório de trabalho atual)
        diretório onde a pasta será criada    
    '''
    try:
        PATH = os.path.join(path, str(folder_name))
        os.mkdir(PATH)
        print("Folder", "{}".format(folder_name), "criado")
        return PATH
    except FileExistsError:
        return PATH
    except TypeError:
        print("TypeError between path ({}) and folder_name ({})".format(path, folder_name))
    

In [2]:
def define_forces(traction_set: set,
                  m: float,
                  m1: float,
                  mu: float=0.42,
                  beta: float=0.02,
                  g: float=9.80665):
    '''
    Função auxiliar que visa simplificar a geração das forças de tração ou reação
    a depender do set de input
    
    Parameters
    ----------
    traction_set: set[int]
        um set de inteiros que simbolizam o tracionamento das rodas dianteiras e
        traseiras; se o valor do elemento é 1, isto quer dizer que o par de rodas
        desta posição é tracionado; se é 0, é não tracionado
    m: float
        valor que representa a massa concentrada da parte dianteira do carro
    m1: float
        valor que representa a massa concentrada da parte traseira do carro
    mu: float (default=0.42)
        valor que representa o coeficiente de atrito dinâmico entre o pneu e o
        asfalto para o caso de rolamento (ativo quando há tração)
    beta: float (default=0.02)
        valor que representa o coeficiente de atrito dinâmico entre o pneu e o
        asfalto para o caso de escorregamento (ativo quando não há tração)
    g: float (default=9.80665)
        valor que representa a aceleração da gravidade
    
    Returns
    -------
    F: float
        força, de tração (negativa no referencial) ou de reação (positiva no referencial),
        resultante nas rodas dianteiras
    F1: float
        força, de tração (positiva no referencial) ou de reação (negativa no referencial),
        resultante nas rodas traseiras
    
    Examples
    --------
    >>>define_forces((0, 1), 1000, 939) # -> implica que a tração é traseira
    >>>define_forces((1, 1), 1000, 939) # -> implica que a tração é nas quatro rodas
    >>>define_forces((1, 0), 1000, 939) # -> implica que a tração é dianteira
    '''
    diant, tras = traction_set
    # bloco de análise da parte dianteira
    if diant == 0:
        F = beta * m * g
    else:
        F = - mu * m * g # negativo para seguir referencial dado
    # bloco de análise da parte traseira
    if tras == 0:
        F1 = - beta * m1 * g
    else:
        F1 = mu * m1 * g
    
    return F, F1

def get_concentrated_masses(r: float, m_total: float=1939):
    '''
    Função que retorna a distribuição de massa nos eixos dianteiro e traseiro
    
    Parameters
    ----------
    r: float
        razão entre a massa do eixo dianteiro e a massa total do carro; input do sistema
    m_total: float (default=1939)
        massa total do sistema
        
    Returns
    -------
    m: float
        massa concentrada da parte dianteira do carro
    m1: float
        massa concentrada da parte traseira do carro
    '''
    m = r * m_total
    m1 = m_total - m
    
    return m, m1

In [3]:
def f1(t, y):
    '''
    Função que calcula, a partir do desenvolvimento das equações fornecidas no
    enunciado, a segunda derivada de theta no passo i usando dos valores de 
    theta e theta ponto neste mesmo passo
    
    Parameters
    ----------
    t: float
        instante de tempo em que a derivada é calculada
    y: np.array (dim=[4,  ])
        uma array unidimensional com 4 valores; vetor de estados do sistema 
        para o passo i
    
    Returns
    --------
    thetapp: float
        valor da segunda derivada de theta calculada para o passo i        
    '''
    # desmembrando as variáveis a partir do vetor de estados
    theta, thetap, x, xp = y
    # por motivos de legibilidade do código e para mitigar possíveis erros,
    # separaremos as partes componentes de thetapp
    # thetapp = A*(B+C+D+E)
    A = np.sin(theta)/(m1*L*(np.sin(theta))**2 - m_total*L)
    B = (2*m_total*I*omega*thetap)/(m1*L*np.sin(theta))
    C = - m1*L*(theta**2)*np.cos(theta)
    D = - F
    E = F1*np.cos(theta)
    thetapp = A*(B + C + D + E)
    
    return thetapp

def f2(t, y):
    '''
    Função que calcula, a partir do desenvolvimento das equações fornecidas no
    enunciado, a segunda derivada de x no passo i usando dos valores de 
    theta, theta ponto e theta dois pontos neste mesmo passo
    
    Parameters
    ----------
    t: float
        instante de tempo em que a derivada é calculada
    y: np.array (dim=[4,  ])
        uma array unidimensional com 4 valores; vetor de estados do sistema 
        para o passo i
    
    Returns
    --------
    xpp: float
        valor da segunda derivada de x calculada para o passo i        
    '''
    # calculando novamente a segunda derivada de theta, importante
    # para tornar `f1` e `f2` funções independentes
    
    # desmembrando as variáveis a partir do vetor de estados
    theta, thetap, x, xp = y
    
    # por motivos de legibilidade do código e para mitigar possíveis erros,
    # separaremos as partes componentes de thetapp
    # thetapp = A*(B+C+D+E)
    A = np.sin(theta)/(m1*L*(np.sin(theta))**2 - m_total*L)
    B = (2*m_total*I*omega*thetap)/(m1*L*np.sin(theta))
    C = - m1*L*(theta**2)*np.cos(theta)
    D = - F
    E = F1*np.cos(theta)
    thetapp = A*(B + C + D + E)
    xpp = - (m1*(L**2)*thetapp + 2*I*omega*thetap)/(m1*L*np.sin(theta))
    
    return xpp

def f2_(t, y):
    '''
    Função de testes de `f2`, gerada a partir da simplificação das equações dinâmicas
    do sistema a partir de outra visão
    
    Parameters
    ----------
    t: float
        instante de tempo em que a derivada é calculada
    y: np.array (dim=[4,  ])
        uma array unidimensional com 4 valores; vetor de estados do sistema 
        para o passo i
    
    Returns
    --------
    xpp: float
        valor da segunda derivada de x calculada para o passo i    
    '''
    # desmembrando as variáveis a partir do vetor de estados
    theta, thetap, x, xp = y
    
    A = np.sin(theta)/(m1*L*(np.sin(theta))**2 - m_total*L)
    B = (2*m_total*I*omega*thetap)/(m1*L*np.sin(theta))
    C = - m1*L*(theta**2)*np.cos(theta)
    D = - F
    E = F1*np.cos(theta)
    thetapp = A*(B + C + D + E)
    xpp = (- F + F1*np.cos(theta) - m1*L*(theta**2*np.cos(theta) + thetapp*np.sin(theta)))/m_total
    
    return xpp

In [4]:
def euler(t0, tmax, h, y0, f1, f2):
    '''
    Função que aplica o método de Euler em um determinado intervalo, com um 
    determinado passo, a partir de um determinado vetor de estados inicial e
    com funções de incremento conhecidas
    
    Parameters
    ----------
    t0: float
        tempo inicial da integração
    tmax: float
        tempo final da integração
    h: float
        passo da integração
    y0: np.array (dim=[1, 4])
        vetor de estados inicial do sistema
    f1: callable
        função que calcula a segunda derivada de theta
    f2: callable
        função que calcula a segunda derivada de x
        
    Returns
    -------
    t: np.array (dim=[N, ])
        vetor de instantes espaçados igualmente entre si (de acordo com o 
        passo h) entre os instantes t0 e tmax
    y: np.array (dim=[N, 4])
        vetor de estados para todos os instantes da simulação
    xpp_array: np.array (dim=[N, 1])
        vetor da segunda derivada de x no tempo, necessária para plotar e mostrar
        os resultados
    '''
    # vetor de instantes; tmax+h para que ele inclua o instante final da
    # simulação
    t = np.arange(t0, tmax+h, h)
    n = t.shape[0]
    
    # vetor de estados y, cada linha representa um instante; em cada linha:
    # primeiro elemento := theta
    # segundo elemento  := thetap
    # terceiro elemento := x
    # quarto elemento   := xp
    y = np.zeros((n, 4))
    
    # setando a primeira linha como o vetor de estados inicial
    y[0, :] = y0
    
    # precisamos também guardar em algum lugar os valores da segunda derivada de
    # x com respeito ao tempo
    xpp_array = np.zeros(n)
    
    # iterando em todos os instantes
    # range começa em 0 pois puxamos os valores da linha 0
    # e termina em n - 2 pois o último valor do vetor é de índice n - 1 e a linha
    # de atualização o faz no índice i + 1; portanto quando i = n - 2 (último
    # índice a ser acessado), i + 1 = n - 1, e portanto estaremos atualizando
    # o vetor do último instante
    for i in range(n - 1):
        thetap  = y[i, 1]
        xp      = y[i, 3]
        thetapp = f1(t[i], y[i, :])
        xpp     = f2(t[i], y[i, :])
        # adicionar xpp a array de xpp's
        xpp_array[i] = xpp
        # com os incrementos faz-se um vetor
        dy = np.array([thetap, thetapp, xp, xpp])
        # aplicando os incrementos ao vetor de estados
        y[i+1, :] = y[i, :] + h*dy
    # adicionar o valor da aceleração do passo n - 1
    xpp     = f2(t[n - 1], y[n - 1, :])
    xpp_array[n - 1] = xpp
    
    return t, y, xpp_array

def RK2(t0, tmax, h, y0, f1, f2):
    '''
    Função que aplica o método de Runge-Kutta de 2a ordem em um determinado 
    intervalo, com um determinado passo, a partir de um determinado vetor de
    estados inicial e com funções de incremento conhecidas
    
    Parameters
    ----------
    t0: float
        tempo inicial da integração
    tmax: float
        tempo final da integração
    h: float
        passo da integração
    y0: np.array (dim=[1, 4])
        vetor de estados inicial do sistema
    f1: callable
        função que calcula a segunda derivada de theta
    f2: callable
        função que calcula a segunda derivada de x
        
    Returns
    -------
    t: np.array (dim=[N, ])
        vetor de instantes espaçados igualmente entre si (de acordo com o 
        passo h) entre os instantes t0 e tmax
    y: np.array (dim=[N, 4])
        vetor de estados para todos os instantes da simulação
    xpp_array: np.array (dim=[N, 1])
        vetor da segunda derivada de x no tempo, necessária para plotar e mostrar
        os resultados
    '''
    # vetor de instantes; tmax+h para que ele inclua o instante final da
    # simulação
    t = np.arange(t0, tmax+h, h)
    n = t.shape[0]
    
    # vetor de estados y, cada linha representa um instante; em cada linha:
    # primeiro elemento := theta
    # segundo elemento  := thetap
    # terceiro elemento := x
    # quarto elemento   := xp
    y = np.zeros((n, 4))
    
    # setando a primeira linha como o vetor de estados inicial
    y[0, :] = y0 
    
    # precisamos também guardar em algum lugar os valores da segunda derivada de
    # x com respeito ao tempo
    xpp_array = np.zeros(n)
    
    # iterando em todos os instantes
    # range começa em 0 pois puxamos os valores da linha 0
    # e termina em n - 2 pois o último valor do vetor é de índice n - 1 e a linha
    # de atualização o faz no índice i + 1; portanto quando i = n - 2 (último
    # índice a ser acessado), i + 1 = n - 1, e portanto estaremos atualizando
    # o vetor do último instante
    for i in range(n - 1):
        # calculando os coeficientes da array `k1`
        k1_thetap  = y[i, 1]
        k1_xp      = y[i, 3]
        k1_thetapp = f1(t[i], y[i, :])
        k1_xpp     = f2(t[i], y[i, :])
        # a segunda derivada de x neste método é o calculado em k1
        xpp_array[i] = k1_xpp
        
        k1 = np.array([k1_thetap, k1_thetapp, k1_xp, k1_xpp])
        
        # calculando os coeficientes da array `k2`
        # calculo do estado num instante intermediário: yi
        yi = y[i, :] + h*k1
        
        k2_thetap  = yi[1]
        k2_xp      = yi[3]
        k2_thetapp = f1(t[i] + h, yi)
        k2_xpp     = f2(t[i] + h, yi)
        
        k2 = np.array([k2_thetap, k2_thetapp, k2_xp, k2_xpp])
        
        # calculo dos valores do proximo passo a partir de `k1` e `k2`
        y[i+1, :] = y[i, :] + (h/2)*(k1 + k2)
        
    # adicionar o valor da aceleração do passo n - 1
    xpp     = f2(t[n - 1], y[n - 1, :])
    xpp_array[n - 1] = xpp
    
    return t, y, xpp_array

def RK4(t0, tmax, h, y0, f1, f2):
    '''
    Função que aplica o método de Runge-Kutta de 4a ordem em um determinado 
    intervalo, com um determinado passo, a partir de um determinado vetor de
    estados inicial e com funções de incremento conhecidas
    
    Parameters
    ----------
    t0: float
        tempo inicial da integração
    tmax: float
        tempo final da integração
    h: float
        passo da integração
    y0: np.array (dim=[1, 4])
        vetor de estados inicial do sistema
    f1: callable
        função que calcula a segunda derivada de theta
    f2: callable
        função que calcula a segunda derivada de x
        
    Returns
    -------
    t: np.array (dim=[N, ])
        vetor de instantes espaçados igualmente entre si (de acordo com o 
        passo h) entre os instantes t0 e tmax
    y: np.array (dim=[N, 4])
        vetor de estados para todos os instantes da simulação
    xpp_array: np.array (dim=[N, 1])
        vetor da segunda derivada de x no tempo, necessária para plotar e mostrar
        os resultados
    '''
    # vetor de instantes; tmax+h para que ele inclua o instante final da
    # simulação
    t = np.arange(t0, tmax+h, h)
    n = t.shape[0]
    
    # vetor de estados y, cada linha representa um instante; em cada linha:
    # primeiro elemento := theta
    # segundo elemento  := thetap
    # terceiro elemento := x
    # quarto elemento   := xp
    y = np.zeros((n, 4))
    
    # setando a primeira linha como o vetor de estados inicial
    y[0, :] = y0
    
    # precisamos também guardar em algum lugar os valores da segunda derivada de
    # x com respeito ao tempo
    xpp_array = np.zeros(n)
    
    # iterando em todos os instantes
    # range começa em 0 pois puxamos os valores da linha 0
    # e termina em n - 2 pois o último valor do vetor é de índice n - 1 e a linha
    # de atualização o faz no índice i + 1; portanto quando i = n - 2 (último
    # índice a ser acessado), i + 1 = n - 1, e portanto estaremos atualizando
    # o vetor do último instante
    for i in range(n - 1):
        # calculando os coeficientes da array `k1`
        k1_thetap  = y[i, 1]
        k1_xp      = y[i, 3]
        k1_thetapp = f1(t[i], y[i, :])
        k1_xpp     = f2(t[i], y[i, :])
        # a segunda derivada de x neste método é o calculado em k1
        xpp_array[i] = k1_xpp
        
        k1 = np.array([k1_thetap, k1_thetapp, k1_xp, k1_xpp])
        
        # calculando os coeficientes da array `k2`
        # calculo do estado num instante intermediário: yik2
        yik2 = y[i, :] + (h/2)*k1
        
        k2_thetap  = yik2[1]
        k2_xp      = yik2[3]
        k2_thetapp = f1(t[i] + h/2, yik2)
        k2_xpp     = f2(t[i] + h/2, yik2)
        
        k2 = np.array([k2_thetap, k2_thetapp, k2_xp, k2_xpp])
        
        # calculando os coeficientes da array `k3`
        # calculo do estado num instante intermediário: yik3
        yik3 = y[i, :] + (h/2)*k2
        
        k3_thetap  = yik3[1]
        k3_xp      = yik3[3]
        k3_thetapp = f1(t[i] + h/2, yik3)
        k3_xpp     = f2(t[i] + h/2, yik3)
        
        k3 = np.array([k3_thetap, k3_thetapp, k3_xp, k3_xpp])
        
        # calculando os coeficientes da array `k4`
        # calculo do estado num instante intermediário: yik4
        yik4 = y[i, :] + h*k3
        
        k4_thetap  = yik4[1]
        k4_xp      = yik4[3]
        k4_thetapp = f1(t[i] + h, yik4)
        k4_xpp     = f2(t[i] + h, yik4)
        
        k4 = np.array([k4_thetap, k4_thetapp, k4_xp, k4_xpp])
        
        # calculo dos valores do proximo passo a partir de `k1`, `k2`, `k3`, e `k4`
        y[i+1, :] = y[i, :] + (h/6)*(k1 + 2*k2 + 2*k3 + k4)
        
    # adicionar o valor da aceleração do passo n - 1
    thetapp = f1(t[n - 1], y[n - 1, :])
    xpp     = f2(t[n - 1], y[n - 1, :])
    xpp_array[n - 1] = xpp
        
    return t, y, xpp_array

def run_method_plot_results(h, title, t0, tmax, y0, f1, f2,
                            method, savefig=True):
    '''
    Método que aplica o método requisitado e devolve as figuras (uma para a 
    variável theta e outra para a variável x) com os resultados das integrações.
    
    Parameters
    ----------
    h: float
        passo da integração
    title: str
        título principal dos gráficos a serem plotados; algo do tipo '1a', '2c'
    t0: float
        tempo inicial da integração
    tmax: float
        tempo final da integração
    y0: np.array (dim=[1, 4])
        vetor de estados inicial do sistema
    f1: callable
        função que calcula a segunda derivada de theta
    f2: callable
        função que calcula a segunda derivada de x
    method: callable
        método de integração numérica; algo do tipo `euler`, `RK2`, `RK4`
    savefig: bool (default=True)
        se a figura gerada na simulação será salva ou apenas mostrada
    '''
    t, y, xpp_array = method(t0, tmax, h, y0, f1, f2)
    
    thetav = y[:, :2]
    # os vetores de x devem ser xp (elemento de índice 3 de y) e xpp (guardado
    # na xpp_array) e deve ser transposta para ter a mesma dimensão de thetav
    xv     = np.vstack([y[:, 3], xpp_array]).T
    
    # titulo e subtitulo das figuras
    method_name = {euler: 'Euler', RK2: 'Runge-Kutta 2', RK4: 'Runge-Kutta 4'}[method]
    title_string  = f'Questão {title}'
    subtitle_string = rf'Resultado da EDO pelo método de {method_name} e $h={h}$'
    
    # bloco da figura de theta
    fig = plt.figure(dpi=200)
    plt.plot(t, thetav[:, 0], label=r'$\theta$', color=cores['theta'])
    ax = plt.gca()
    ax.set_xlabel(r'$t$ (segundos)')
    ax.set_ylabel(r'$\theta$ (rad)')
    ax_ = ax.twinx()
    ax_.set_ylabel(r'$\dot{\theta}$ (rad/s)')
    ax_.plot(t, thetav[:, 1], label=r'$\dot{\theta}$', color=cores['thetap'])

    # como colocamos dois eixos distintos, coletar os `handles` e `labels` para
    # montar uma legenda única
    handles,labels = [],[]
    for ax in fig.axes:
        for handle, label in zip(*ax.get_legend_handles_labels()):
            handles.append(handle)
            labels.append(label)

    plt.legend(handles,labels)
    plt.suptitle(title_string, y=1.0, fontsize = 18)
    ax.set_title(subtitle_string, fontsize = 14)
    
    if savefig:
        FOLDER = create_folder('figures')
        SAVEPATH = os.path.join(FOLDER, f'{title}_theta_{h}.png')
        plt.savefig(SAVEPATH, dpi=200, bbox_inches='tight')
        plt.close(fig)
    else:
        plt.show(fig)
    
    # bloco da figura de x
    fig = plt.figure(dpi=200)
    plt.plot(t, xv[:, 0], label=r'$\dot{x}$', color=cores['xp'])
    ax = plt.gca()
    ax.set_xlabel(r'$t$ (segundos)')
    ax.set_ylabel(r'$\dot{x}$ (m/s)')
    ax_ = ax.twinx()
    ax_.set_ylabel(r'$\ddot{x}$ (m/s²)')
    ax_.plot(t, xv[:, 1], label=r'$\ddot{x}$', color=cores['xpp'])

    # como colocamos dois eixos distintos, coletar os `handles` e `labels` para
    # montar uma legenda única
    handles,labels = [],[]
    for ax in fig.axes:
        for handle, label in zip(*ax.get_legend_handles_labels()):
            handles.append(handle)
            labels.append(label)

    plt.legend(handles,labels)
    plt.suptitle(title_string, y=1.0, fontsize = 18)
    ax.set_title(subtitle_string, fontsize = 14)
    
    if savefig:
        FOLDER = create_folder('figures')
        SAVEPATH = os.path.join(FOLDER, f'{title}_x_{h}.png')
        plt.savefig(SAVEPATH, dpi=200, bbox_inches='tight')
        plt.close(fig)
    else:
        plt.show(fig)
        
def plot_var_varp(t0, tmax, h, y0, f1, f2, method, title, savefig=True):
    '''
    Função que plota, para os parâmetros passados, o gráfico da derivada de 
    uma variável em função da própria variável
    
    Parameters
    ----------
    t0: float
        tempo inicial da integração
    tmax: float
        tempo final da integração
    h: float
        passo da integração
    y0: np.array (dim=[1, 4])
        vetor de estados inicial do sistema
    f1: callable
        função que calcula a segunda derivada de theta
    f2: callable
        função que calcula a segunda derivada de x
    method: callable
        método de integração numérica; algo do tipo `euler`, `RK2`, `RK4`
    title: str
        titulo do plot gerado
    savefig: bool (default=True)
        se a figura deve ou não ser salva
    '''
    t, y, xpp_array = method(t0, tmax, h, y0, f1, f2)
    
    thetav = y[:, :2]
    # os vetores de x devem ser xp (elemento de índice 3 de y) e xpp (guardado
    # na xpp_array) e deve ser transposta para ter a mesma dimensão de thetav
    xv     = np.vstack([y[:, 3], xpp_array]).T
    
    # titulo e subtitulo das figuras
    method_name = {euler: 'Euler', RK2: 'Runge-Kutta 2', RK4: 'Runge-Kutta 4'}[method]
    subtitle_string = rf'A partir do método de {method_name} e $h={h}$'
    
    # para theta
    plt.plot(thetav[:, 0], thetav[:, 1], label=r'$\dot{\theta}\;(\theta)$', color=cores['thetap'], lw=1)
    plt.grid()
    ax, fig = plt.gca(), plt.gcf()
    ax.set_xlabel(r'$\theta$ (rad)')
    ax.set_ylabel(r'$\dot{\theta}$ (rad/s)')
    plt.scatter(thetav[0, 0], thetav[0, 1], label='Posição inicial', marker='x', color=colors[16])
    plt.legend(loc='upper right')
    title_string  = f'Questão {title}' + r' - Gráfico de $\dot{\theta}$ em função de $\theta$'
    plt.suptitle(title_string, y=1.0, fontsize = 18)
    ax.set_title(subtitle_string, fontsize = 14)
    
    if savefig:
        FOLDER = create_folder('figures')
        SAVEPATH = os.path.join(FOLDER, f'fases_{title}_theta_{h}.png')
        plt.savefig(SAVEPATH, dpi=200, bbox_inches='tight')
        plt.close(fig)
    else:
        plt.show(fig)
    
    # para x
    plt.plot(xv[:, 0], xv[:, 1], label=r'$\ddot{x}\;(\dot{x})$', color=cores['xpp'], lw=1)
    plt.grid()
    ax, fig = plt.gca(), plt.gcf()
    ax.set_xlabel(r'$\dot{x}$ (m/s)')
    ax.set_ylabel(r'$\ddot{x}$ (m/s²)')
    plt.scatter(xv[0, 0], xv[0, 1], label='Posição inicial', marker='x', color=colors[16])
    plt.legend(loc='upper right')
    title_string  = f'Questão {title}' + r' - Gráfico de $\ddot{x}$ em função de $\dot{x}$'
    plt.suptitle(title_string, y=1.0, fontsize = 18)
    ax.set_title(subtitle_string, fontsize = 14)
    
    if savefig:
        FOLDER = create_folder('figures')
        SAVEPATH = os.path.join(FOLDER, f'fases_{title}_x_{h}.png')
        plt.savefig(SAVEPATH, dpi=200, bbox_inches='tight')
        plt.close(fig)
    else:
        plt.show(fig)
        
def compare_methods(t0, tmax, h, y0, f1, f2, savefig=True):
    '''
    Função que, para um dado valor de h, simula o sistema de EDOs dado
    (a partir de t0, tmax, y0, f1, e f2) e plota as variáveis resultantes
    em plots separados para comparação dos métodos empregados
    
    Parameters
    ----------
    t0: float
        tempo inicial da integração
    tmax: float
        tempo final da integração
    h: float
        passo da integração
    y0: np.array (dim=[1, 4])
        vetor de estados inicial do sistema
    f1: callable
        função que calcula a segunda derivada de theta
    f2: callable
        função que calcula a segunda derivada de x
    savefig: bool (default=True)
        se a figura deve ou não ser salva
    '''
    def plot_var(t, var_array, var_name, savefig):
        '''
        Função auxiliar, plota os métodos e salva as figuras
        '''
        # cores de cada um dos métodos pro plot individual comparativo; a ordem escolhida
        # para serem plotados é `euler`, `RK2`, e então `RK4`
        methods_colors = [colors[2], colors[10], colors[16]]
        labels = ['Euler', 'Runge-Kutta 2', 'Runge-Kutta 4']
        var_latex = {'theta': r'$\theta$', 'thetap': r'$\dot{\theta}$', 'xp': r'$\dot{x}$',
                    'xpp': r'$\ddot{x}$'}
        var_unit  = {'theta': 'rad', 'thetap': 'rad/s', 'xp': 'm/s',
                    'xpp': 'm/s²'}

        title = 'Comparação dos métodos aplicados para a variável'
        subtitle = rf'Análise de {var_latex[var_name]}' + f' com $h={h}$'

        ax, fig = plt.gca(), plt.gcf()
        ax.set_prop_cycle(color=methods_colors)
        plt.plot(t, theta)
        plt.legend(labels=labels)
        ax.set_xlabel(r'$t$ (segundos)')
        ax.set_ylabel(rf'{var_latex[var_name]} ({var_unit[var_name]})')

        plt.suptitle(title, y=1.0, fontsize = 18)
        ax.set_title(subtitle, fontsize = 14)
        plt.grid()

        if savefig:
            FOLDER = create_folder('figures')
            SAVEPATH = os.path.join(FOLDER, f'metodos_{var_name}_{h}.png')
            plt.savefig(SAVEPATH, dpi=200, bbox_inches='tight')
            plt.close(fig)
        else:
            plt.show(fig)
            
    t, y_euler, xpp_array_euler = euler(t0, tmax, h, y0, f1, f2)
    t, y_RK2  , xpp_array_RK2   =   RK2(t0, tmax, h, y0, f1, f2)
    t, y_RK4  , xpp_array_RK4   =   RK4(t0, tmax, h, y0, f1, f2)
    
    # separa as variáveis em novos vetores
    theta  = np.vstack([y_euler[:, 0], y_RK2[:, 0], y_RK4[:, 0]]).T
    thetap = np.vstack([y_euler[:, 1], y_RK2[:, 1], y_RK4[:, 1]]).T
    xp     = np.vstack([y_euler[:, 3], y_RK2[:, 3], y_RK4[:, 3]]).T
    xpp    = np.vstack([xpp_array_euler, xpp_array_RK2, xpp_array_RK4]).T
    
    plot_var(t, theta,  'theta',  savefig)
    plot_var(t, thetap, 'thetap', savefig)
    plot_var(t, xp,     'xp',     savefig)
    plot_var(t, xpp,    'xpp',    savefig)

In [5]:
# bloco de constantes
m_total = 1939 #....... kg
L       = 2.95 #........ m
I       =    1 #... kg.m^2
omega   =   10 #.... rad/s
theta0  =   10 #........ °
thetap0 =    0 #.... rad/s
x0      =    0 #........ m
xp0     =    0 #...... m/s

# transformar theta0 de graus para radianos
theta0 = np.radians(theta0)

In [6]:
# primeiro experimento com todos os métodos integrativos
# m = 0.6 m_total; m1 = 0.4 m_total
# tração traseira
# ratio entre a massa concentrada do eixo dianteiro `m` e a massa total `m_total`
# para o desenvolvimento dos métodos numéricos
r = 0.6
m, m1 = get_concentrated_masses(r, m_total=m_total)
F, F1 = define_forces((0, 1), m, m1)

# vetor de estado inicial
y0 = np.array([theta0, thetap0, x0, xp0])
t0, tmax = 0, 20
h_list = [1e-1, 1e-2, 1e-3, 1e-4]
methods_list = [euler, RK2, RK4]

for i, method in enumerate(methods_list):
    # nomeação dos experimentos
    # a: Euler
    # b: RK2
    # c: RK4
    title = '1'+chr(ord('a') + i)
    for h in h_list:
        run_method_plot_results(h, title, t0, tmax, y0, f1, f2, method)

# plot das derivadas em função das variáveis, para facilitar o entendimento
# do problema
plot_var_varp(t0, tmax, h, y0, f1, f2, RK4, '1')

# plot de comparação dos três métodos para todos os valores de h
for h in h_list:
    compare_methods(t0, tmax, h, y0, f1, f2)
        
# segundo experimento, usando apenas RK4, razão de massas diferente, tração
# do carro em locais diferentes; h fixo em 1e-4; t0 e tmax fixos em 0 e 20;
# y0 o mesmo do exemplo anterior
# esta lista de listas guarda as informações sobre as razões de massa
RK4_tests = [
    [(0, 1), 0.8],
    [(1, 0), 0.8],
    [(1, 0), 0.2],
    [(0, 1), 0.2],
    [(1, 1), 0.2],
    [(1, 1), 0.8]
]
method =  RK4 #... método
h      = 1e-4 #.... passo
t0     =    0 #........ s
tmax   =   20 #........ s

# vetor de estado inicial
y0 = np.array([theta0, thetap0, x0, xp0])

for i, element in enumerate(RK4_tests):
    m, m1 = get_concentrated_masses(element[1])
    F, F1 = define_forces(element[0], m, m1)
    title = '2'+chr(ord('a') + i)
    run_method_plot_results(h, title, t0, tmax, y0, f1, f2, method)
    plot_var_varp(t0, tmax, h, y0, f1, f2, method, title)

Folder figures criado
